In [ ]:
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.7 MB/s eta 0:00:00


In [5]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-14 16:08:20.079589: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763136500.262752      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763136500.315676      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [7]:
!pip install -q peft

In [ ]:
from peft import PeftModel

In [8]:
!pip install -q accelerate bitsandbytes

In [9]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 10.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.4 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [15]:
# ============================================================
# 1. Set Kaggle dataset path
# ============================================================
ADAPTER_DIR = "/kaggle/input/my-finetuned-llm"   # Replace with your MAK if needed

In [16]:
import torch

In [17]:
# ============================================================
# 2. Load 4-bit base model
# ============================================================
BASE_ID = "unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    BASE_ID,
    max_seq_length = 2048,
    load_in_4bit = True,
)
print("=" * 50)
print("Base model loaded")
print("=" * 50)

# ============================================================
# 3. Load LoRA adapter
# ============================================================
model = PeftModel.from_pretrained(model, ADAPTER_DIR)
print("=" * 50)
print("LoRA loaded")
print("=" * 50)

# ============================================================
# 4. Switch to inference mode
# ============================================================
FastLanguageModel.for_inference(model)
print("=" * 50)
print("Model ready for inference")
print("=" * 50)

model.eval()
print("\n\n")
print("=" * 50)
print("Fine-tuned model is ready!")
print("=" * 50)

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Base model loaded


/usr/local/lib/python3.11/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['target_parameters'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


LoRA loaded
Model ready for inference



Fine-tuned model is ready!


In [18]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))
print("Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("Reserved:", torch.cuda.memory_reserved() / 1024**2, "MB")


CUDA available: True
GPU name: Tesla T4
Allocated: 11855.986328125 MB
Reserved: 12032.0 MB


In [20]:
import gradio as gr
import torch
import re
from threading import Lock

# -------------------------
# YOUR MODEL + TOKENIZER
# -------------------------
tok = tokenizer
mdl = model
mdl.eval()

gen_lock = Lock()

# -------------------------
# Format the model output
# -------------------------
def format_output(raw):
    sections = {
        "verdict": r"(?i)verdict[:\-]\s*(.*)",
        "score": r"(?i)score[:\-]\s*([0-9\.]+)",
        "benefits": r"(?i)benefits?[:\-](.*?)(?=risks?:|overall|conclusion|$)",
        "risks": r"(?i)risks?[:\-](.*?)(?=overall|conclusion|$)",
        "overall": r"(?i)(overall|rationale|analysis)[:\-](.*)",
    }

    formatted = {}
    for key, pattern in sections.items():
        m = re.search(pattern, raw, re.S)
        if m:
            text = m.group(1).strip()
        else:
            text = ""
        formatted[key] = text

    return formatted


# -------------------------
# Evaluation Function
# -------------------------
history = []

def evaluate_story(story, context):
    with gen_lock:
        if not story or story.strip() == "":
            return "Please enter a story.", 0, history

        template = """Below is an instruction that describes a task.

### Instruction:
Evaluate whether the following idea uplifts humanity. Provide:
- Verdict (Yes/No or Positive/Negative)
- Score from 1 to 5 (integer number)
- Benefits
- Risks
- Overall rationale
Be objective and specific.

### Input:
{story}

### Additional Context:
{context}

### Response:
"""

        prompt = template.format(
            story = story.strip(),
            context = context.strip() if context else "None"
        )

        inputs = tok(prompt, return_tensors="pt").to("cuda")

        outputs = mdl.generate(
            **inputs,
            max_new_tokens = 400,
            temperature   = 0.4,
            top_p         = 0.95,
            do_sample     = True,
        )

        raw_text = tok.decode(outputs[0], skip_special_tokens=True)

        # Clean prefix if exists
        if "### Response:" in raw_text:
            raw_text = raw_text.split("### Response:")[1].strip()

        parsed = format_output(raw_text)

        # Score handling
        try:
            score = int(parsed["score"])
        except:
            score = 0

        # Save history
        history.append({
            "story": story,
            "score": score,
            "verdict": parsed["verdict"],
            "output": raw_text[:600] + ("..." if len(raw_text) > 600 else "")
        })

        display_text = f"""
### Verdict
{parsed["verdict"]}

### Score
{parsed["score"]}

### Benefits
{parsed["benefits"]}

### Risks
{parsed["risks"]}

### Overall Rationale
{parsed["overall"]}
""".strip()

        return display_text, score, history


# -------------------------
# Gradio UI
# -------------------------
with gr.Blocks() as demo:
    gr.Markdown("# 🌍 Humanity Uplift Evaluator — Kaggle Demo")
    gr.Markdown("Enter an idea / story. The model evaluates whether it uplifts humanity.")

    with gr.Row():
        story = gr.Textbox(
            label="Story / Idea",
            placeholder="Enter the narrative or idea here...",
            lines=8
        )
        context = gr.Textbox(
            label="Additional Context (optional)",
            placeholder="Studio mission, values, situation...",
            lines=4
        )

    btn = gr.Button("Evaluate", variant="primary")

    with gr.Row():
        output = gr.Markdown(label="Model Output")
        score_bar = gr.Slider(label="Score ( to 5)", minimum=1, maximum=5, value=1, interactive=False)

    history_box = gr.JSON(label="History (previous evaluations)")

    btn.click(
        evaluate_story,
        inputs=[story, context],
        outputs=[output, score_bar, history_box]
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://9a48064cda7da857ee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Make sure if Gradio is working as expected

# After running Gradio, run:
import torch

print("GPU:", torch.cuda.get_device_name(0))
print("Memory allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")